# 1. 기본 설정 / 경로 설정

In [ ]:
# Cell 1: 기본 설정 및 경로 자동 탐색

from pathlib import Path
import csv
import subprocess
import shutil
import pandas as pd

# Vina 실행 파일 이름 (환경에 맞게 수정)
# 예: 리눅스/WSL: "vina", 윈도우: r"C:\Program Files\AutoDock_Vina\vina.exe"
VINA_CMD = "vina"

# 이번에 디버깅할 run 폴더 ID
RUN_ID = "PDP_20251122_221946"  # 다른 run 테스트할 땐 여기만 바꿔주면 됨

# 1) 현재 작업 디렉토리 확인
cwd = Path.cwd()
print("현재 작업 디렉토리:", cwd)

# 2) cwd 및 부모 폴더들에서 'pipeline' 폴더 자동 탐색
pipeline_root = None

for base in [cwd] + list(cwd.parents):
    # case 1: base 밑에 pipeline 폴더가 있는 경우 (ex. 프로젝트 루트에서 노트북 실행)
    if (base / "pipeline").is_dir():
        pipeline_root = base / "pipeline"
        break
    # case 2: base 자체가 pipeline 폴더인 경우 (ex. pipeline 폴더 안에서 노트북 실행)
    if base.name == "pipeline":
        pipeline_root = base
        break

if pipeline_root is None:
    print("[ERROR] 현재 경로 기준으로 'pipeline' 폴더를 찾지 못했습니다.")
    print("→ pipeline 폴더의 절대 경로를 직접 지정해주세요.")
    print("예) pipeline_root = Path(r'C:/Users/xxx/pepbind_project/pipeline')")
else:
    print("pipeline_root:", pipeline_root)

# 3) RUN_ID 기준 하위 폴더 설정
run_dir = (pipeline_root / RUN_ID) if pipeline_root is not None else None

if run_dir is not None and run_dir.exists():
    fasta_dir = run_dir / "fasta"
    pdb_dir = run_dir / "pdb"
    colabfold_dir = pdb_dir / "colabfold_output"
    results_dir = run_dir / "results"
    temp_dir = run_dir / "temp"

    print("\n[폴더 존재 여부 체크]")
    for p in [run_dir, fasta_dir, pdb_dir, colabfold_dir, results_dir, temp_dir]:
        print(p, ":", "OK" if p.exists() else "NOT FOUND")
else:
    print(f"[ERROR] run_dir를 찾지 못했습니다: {run_dir}")
    print("→ RUN_ID가 맞는지, pipeline_root 아래에 해당 폴더가 있는지 한 번 확인해주세요.")

현재 작업 디렉토리: /home/aisys/work/pipeline
pipeline_root: /home/aisys/work/pipeline

[폴더 존재 여부 체크]
/home/aisys/work/pipeline/PDP_20251122_221946 : OK
/home/aisys/work/pipeline/PDP_20251122_221946/fasta : OK
/home/aisys/work/pipeline/PDP_20251122_221946/pdb : OK
/home/aisys/work/pipeline/PDP_20251122_221946/pdb/colabfold_output : OK
/home/aisys/work/pipeline/PDP_20251122_221946/results : OK
/home/aisys/work/pipeline/PDP_20251122_221946/temp : OK


# 2. ColabFold 결과에서 rank1 복합체 PDB 찾기

In [3]:
# Cell 2: ColabFold 출력에서 rank1 복합체 PDB 찾기

def find_rank1_complex_pdbs(colabfold_dir: Path):
    """
    colabfold_output 폴더 안에서 '복합체'로 추정되는 rank1 PDB를 찾는다.
    - 실제 파일명 규칙에 맞게 is_rank1_complex 부분을 필요하면 수정.
    """
    if not colabfold_dir.exists():
        raise FileNotFoundError(f"ColabFold 출력 폴더가 없습니다: {colabfold_dir}")

    pdb_files = sorted(colabfold_dir.rglob("*.pdb"))
    print(f"colabfold_output 내 전체 PDB 개수: {len(pdb_files)}")

    def is_rank1_complex(p: Path) -> bool:
        name = p.name.lower()
        # 예시 규칙:
        #  - 파일명에 'complex' 포함
        #  - 그리고 'rank' 또는 '001' 같은 랭킹 키워드 포함
        # 네 프로젝트 파일명이 다르면 여기 조건을 바꿔줘.
        if "complex" in name and ("rank" in name or "001" in name):
            return True
        return False

    rank1 = [p for p in pdb_files if is_rank1_complex(p)]
    print(f"rank1 복합체 후보 PDB: {len(rank1)}개")

    # 샘플 몇 개만 출력해서 경로 확인
    for p in rank1[:10]:
        print(" -", p.relative_to(colabfold_dir))

    return rank1


rank1_pdbs = find_rank1_complex_pdbs(colabfold_dir)
len(rank1_pdbs)

colabfold_output 내 전체 PDB 개수: 10
rank1 복합체 후보 PDB: 10개
 - complex_0_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb
 - complex_1_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb
 - complex_2_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb
 - complex_3_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb
 - complex_4_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb
 - complex_5_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb
 - complex_6_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb
 - complex_7_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb
 - complex_8_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb
 - complex_9_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb


10

# 3. 기존 유틸 함수 import (split_complex_to_receptor_ligand 등)

In [4]:
# Cell 3: 기존 파이프라인 유틸 함수 import
# 실제 네 프로젝트에서 이 함수들이 정의된 .py 파일 이름으로 수정 필요

from pepbind_pipeline import (
    split_complex_to_receptor_ligand,
    prepare_pdbqt,
    compute_box_from_ligand,
    parse_vina_score_from_stdout,
)

print("유틸 함수 import 완료")

/home/aisys/miniconda3/envs/pepbind/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[INFO] PyTorch device: cuda
유틸 함수 import 완료


# 4. 체인 B 없는 PDB는 스킵하도록 수정한 run_vina_on_rank1

In [5]:
# Cell 4: 체인 분리 에러를 안전하게 처리하는 run_vina_on_rank1

def run_vina_on_rank1(rank1_pdbs, vina_dir: Path):
    """
    AutoDock Vina 도킹 실행 + 요약/상태 로그 기록.

    - vina_summary.csv 컬럼:
        complex, vina_score, vina_status, receptor_pdbqt, ligand_pdbqt, log_file
    - vina_status 예시:
        '정상', '정상(점수=0.0)',
        '실패: Vina 실행 에러(code=...)',
        '실패: Vina 실행 에러(code=...)(PDBQT parsing error: ...)',
        '파싱실패: stdout에서 점수 패턴 없음',
        '스킵: 체인 분리 실패(체인 B 없음 등)',
        '실패: Vina 준비 단계 에러(...)'
    """
    print("\n" + "=" * 80)
    print("STEP 4: AutoDock Vina 도킹 (기존 ColabFold 결과에서 재시작)")
    print("=" * 80)

    if not rank1_pdbs:
        print("[WARN] Vina 실행할 rank1 PDB가 없습니다.")
        return
    if not shutil.which(VINA_CMD):
        print(f"[WARN] VINA_CMD='{VINA_CMD}' 실행 파일을 찾을 수 없습니다. (PATH 확인 필요)")
        return

    vina_dir.mkdir(parents=True, exist_ok=True)
    summary_rows = [["complex", "vina_score", "vina_status", "receptor_pdbqt", "ligand_pdbqt", "log_file"]]
    debug_lines = []

    for complex_pdb in rank1_pdbs:
        base = complex_pdb.stem
        complex_out_dir = vina_dir / base
        complex_out_dir.mkdir(parents=True, exist_ok=True)

        print(f"\n[INFO] Vina 준비: {complex_pdb.name}")

        # 기본값 (문제가 생겨도 요약에는 한 줄 남기기 위함)
        best_score = None
        status = ""
        rec_pdbqt_name = ""
        lig_pdbqt_name = ""
        log_file_name = ""

        # 1) 리셉터/리간드 체인 분리
        try:
            rec_pdb, lig_pdb = split_complex_to_receptor_ligand(
                complex_pdb,
                complex_out_dir,
                receptor_chain="A",
                ligand_chain="B",
            )
        except Exception as e:
            # 체인 B 없음, 체인 레이블 꼬임 등
            status = f"스킵: 체인 분리 실패({type(e).__name__}: {e})"
            print(f"[WARN] {complex_pdb.name} -> {status}")
            debug_lines.append(f"{base}\t{status}")
            summary_rows.append([base, best_score, status, rec_pdbqt_name, lig_pdbqt_name, log_file_name])
            continue

        # 2) PDBQT 변환 + 박스 계산
        try:
            rec_pdbqt, lig_pdbqt = prepare_pdbqt(rec_pdb, lig_pdb, complex_out_dir)
            rec_pdbqt_name = rec_pdbqt.name
            lig_pdbqt_name = lig_pdbqt.name

            box = compute_box_from_ligand(lig_pdb)

            out_pdbqt = complex_out_dir / f"{base}_vina_out.pdbqt"
            log_file  = complex_out_dir / f"{base}_vina_stdout.txt"
            log_file_name = log_file.name
        except Exception as e:
            status = f"실패: Vina 준비 단계 에러({type(e).__name__}: {e})"
            print(f"[ERROR] {complex_pdb.name} -> {status}")
            debug_lines.append(f"{base}\t{status}")
            summary_rows.append([base, best_score, status, rec_pdbqt_name, lig_pdbqt_name, log_file_name])
            continue

        # 3) Vina 실행
        vina_cmd = (
            f"{VINA_CMD} "
            f"--receptor {rec_pdbqt} "
            f"--ligand {lig_pdbqt} "
            f"--center_x {box['center_x']:.3f} --center_y {box['center_y']:.3f} --center_z {box['center_z']:.3f} "
            f"--size_x {box['size_x']:.3f} --size_y {box['size_y']:.3f} --size_z {box['size_z']:.3f} "
            f"--out {out_pdbqt}"
        )

        print(f"[RUN] {vina_cmd}")
        result = subprocess.run(
            vina_cmd,
            shell=True,
            capture_output=True,
            text=True,
        )

        # 로그 파일 기록
        with open(log_file, "w", encoding="utf-8") as lf:
            lf.write("=== STDOUT ===\n")
            lf.write(result.stdout or "")
            lf.write("\n\n=== STDERR ===\n")
            lf.write(result.stderr or "")

        # 4) 결과 상태 / 점수 파싱
        if result.returncode != 0:
            status = f"실패: Vina 실행 에러(code={result.returncode})"
            if "PDBQT parsing error" in (result.stdout or "") or "PDBQT parsing error" in (result.stderr or ""):
                status += " (PDBQT parsing error: flex residue/ligand 태그 문제)"
            print(f"[ERROR] {status}. 로그 파일: {log_file}")
            print(result.stdout or "")
            print(result.stderr or "")
            debug_lines.append(f"{base}\t{status}\tlog={log_file.name}")
        else:
            best_score = parse_vina_score_from_stdout(result.stdout or "")
            if best_score is None:
                status = "파싱실패: stdout에서 점수 패턴 없음"
                print(f"[WARN] {complex_pdb.name} Vina 점수 파싱 실패. 로그 파일 확인: {log_file}")
                debug_lines.append(f"{base}\t{status}\tlog={log_file.name}")
            else:
                if best_score == 0.0:
                    status = "정상(점수=0.0)"
                else:
                    status = "정상"
                print(f"[INFO] {complex_pdb.name} Vina best score: {best_score}")
                debug_lines.append(f"{base}\t{status}\t{best_score}")

        summary_rows.append([base, best_score, status, rec_pdbqt_name, lig_pdbqt_name, log_file_name])

    # 요약 파일 저장
    summary_csv = vina_dir / "vina_summary.csv"
    with open(summary_csv, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerows(summary_rows)
    print(f"\n✅ Vina 요약 저장: {summary_csv}")

    if debug_lines:
        debug_file = vina_dir / "vina_debug.txt"
        debug_file.write_text("\n".join(debug_lines), encoding="utf-8")
        print(f"[INFO] Vina 디버그 로그: {debug_file}")

    print("=" * 80)

# 5. 기존 ColabFold 결과에서 바로 Vina 단계만 실행

In [6]:
# Cell 5: 기존 ColabFold 결과에서 Vina 단계만 실행

vina_dir = results_dir / "vina"
vina_dir.mkdir(parents=True, exist_ok=True)

print("Vina 결과 저장 폴더:", vina_dir)

# 필요하면 일부만 테스트: rank1_pdbs[:5]
target_pdbs_for_vina = rank1_pdbs  # 또는 rank1_pdbs[:5]

run_vina_on_rank1(target_pdbs_for_vina, vina_dir=vina_dir)

Vina 결과 저장 폴더: /home/aisys/work/pipeline/PDP_20251122_221946/results/vina

STEP 4: AutoDock Vina 도킹 (기존 ColabFold 결과에서 재시작)

[INFO] Vina 준비: complex_0_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb
[RUN] /home/aisys/miniconda3/envs/pepbind/bin/obabel -ipdb /home/aisys/work/pipeline/PDP_20251122_221946/results/vina/complex_0_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000/complex_0_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000_receptor_A.pdb -xr -opdbqt -O /home/aisys/work/pipeline/PDP_20251122_221946/results/vina/complex_0_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000/complex_0_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000_receptor_A.pdbqt
[RUN] /home/aisys/miniconda3/envs/pepbind/bin/obabel -ipdb /home/aisys/work/pipeline/PDP_20251122_221946/results/vina/complex_0_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000/complex_0_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000_ligand_B.pdb      -opdbqt -O /

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is /home/aisys/work/pipeline/PDP_20251122_221946/results/vina/complex_0_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000/complex_0_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000_receptor_A.pdb)

1 molecule converted
1 molecule converted


[RUN] vina --receptor /home/aisys/work/pipeline/PDP_20251122_221946/results/vina/complex_0_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000/complex_0_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000_receptor_A.pdbqt --ligand /home/aisys/work/pipeline/PDP_20251122_221946/results/vina/complex_0_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000/complex_0_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000_ligand_B.pdbqt --center_x 16.155 --center_y 40.828 --center_z -20.157 --size_x 21.429 --size_y 21.031 --size_z 17.375 --out /home/aisys/work/pipeline/PDP_20251122_221946/results/vina/complex_0_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000/complex_0_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000_vina_out.pdbqt
[INFO] complex_0_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb Vina best score: -2.97

[INFO] Vina 준비: complex_1_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb
[RUN] /home/aisys/miniconda3/

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is /home/aisys/work/pipeline/PDP_20251122_221946/results/vina/complex_1_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000/complex_1_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000_receptor_A.pdb)

1 molecule converted
1 molecule converted


[INFO] complex_1_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb Vina best score: -3.651

[INFO] Vina 준비: complex_2_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb
[RUN] /home/aisys/miniconda3/envs/pepbind/bin/obabel -ipdb /home/aisys/work/pipeline/PDP_20251122_221946/results/vina/complex_2_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000/complex_2_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000_receptor_A.pdb -xr -opdbqt -O /home/aisys/work/pipeline/PDP_20251122_221946/results/vina/complex_2_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000/complex_2_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000_receptor_A.pdbqt
[RUN] /home/aisys/miniconda3/envs/pepbind/bin/obabel -ipdb /home/aisys/work/pipeline/PDP_20251122_221946/results/vina/complex_2_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000/complex_2_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000_ligand_B.pdb      -opdbqt -O /home/aisys/work/pipe

...

[INFO] complex_5_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb Vina best score: -4.275

[INFO] Vina 준비: complex_6_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb
[WARN] complex_6_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb -> 스킵: 체인 분리 실패(ValueError: /home/aisys/work/pipeline/PDP_20251122_221946/pdb/colabfold_output/complex_6_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb에서 체인 B 원자를 찾지 못했습니다.)

[INFO] Vina 준비: complex_7_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb
[RUN] /home/aisys/miniconda3/envs/pepbind/bin/obabel -ipdb /home/aisys/work/pipeline/PDP_20251122_221946/results/vina/complex_7_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000/complex_7_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000_receptor_A.pdb -xr -opdbqt -O /home/aisys/work/pipeline/PDP_20251122_221946/results/vina/complex_7_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000/complex_7_unrelaxed_rank_001_alp

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is /home/aisys/work/pipeline/PDP_20251122_221946/results/vina/complex_7_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000/complex_7_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000_receptor_A.pdb)

1 molecule converted
1 molecule converted


[INFO] complex_7_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb Vina best score: -3.866

[INFO] Vina 준비: complex_8_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb
[RUN] /home/aisys/miniconda3/envs/pepbind/bin/obabel -ipdb /home/aisys/work/pipeline/PDP_20251122_221946/results/vina/complex_8_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000/complex_8_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000_receptor_A.pdb -xr -opdbqt -O /home/aisys/work/pipeline/PDP_20251122_221946/results/vina/complex_8_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000/complex_8_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000_receptor_A.pdbqt
[RUN] /home/aisys/miniconda3/envs/pepbind/bin/obabel -ipdb /home/aisys/work/pipeline/PDP_20251122_221946/results/vina/complex_8_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000/complex_8_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000_ligand_B.pdb      -opdbqt -O /home/aisys/work/pipe

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is /home/aisys/work/pipeline/PDP_20251122_221946/results/vina/complex_8_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000/complex_8_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000_receptor_A.pdb)

1 molecule converted
1 molecule converted
*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is /home/aisys/work/pipeline/PDP_20251122_221946/results/vina/complex_9_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000/complex_9_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000_receptor_A.pdb)

1 molecule converted
1 molecule converted


[RUN] /home/aisys/miniconda3/envs/pepbind/bin/obabel -ipdb /home/aisys/work/pipeline/PDP_20251122_221946/results/vina/complex_9_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000/complex_9_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000_ligand_B.pdb      -opdbqt -O /home/aisys/work/pipeline/PDP_20251122_221946/results/vina/complex_9_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000/complex_9_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000_ligand_B.pdbqt
[RUN] vina --receptor /home/aisys/work/pipeline/PDP_20251122_221946/results/vina/complex_9_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000/complex_9_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000_receptor_A.pdbqt --ligand /home/aisys/work/pipeline/PDP_20251122_221946/results/vina/complex_9_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000/complex_9_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000_ligand_B.pdbqt --center_x 4.563 --center_y -11.900 --center_z

# 6. Vina 요약 결과를 판다스로 확인

In [7]:
# Cell 6: vina_summary.csv 로드해서 결과 확인

summary_csv = vina_dir / "vina_summary.csv"
if summary_csv.exists():
    vina_df = pd.read_csv(summary_csv)
    print("요약 파일 경로:", summary_csv)
    display(vina_df.head(20))
else:
    print("vina_summary.csv 를 찾을 수 없습니다:", summary_csv)

요약 파일 경로: /home/aisys/work/pipeline/PDP_20251122_221946/results/vina/vina_summary.csv


,complex,vina_score,vina_status,receptor_pdbqt,ligand_pdbqt,log_file
0,complex_0_unrelaxed_rank_001_alphafold2_multim...,-2.970,정상,complex_0_unrelaxed_rank_001_alphafold2_multim...,complex_0_unrelaxed_rank_001_alphafold2_multim...,complex_0_unrelaxed_rank_001_alphafold2_multim...
1,complex_1_unrelaxed_rank_001_alphafold2_multim...,-3.651,정상,complex_1_unrelaxed_rank_001_alphafold2_multim...,complex_1_unrelaxed_rank_001_alphafold2_multim...,complex_1_unrelaxed_rank_001_alphafold2_multim...
2,complex_2_unrelaxed_rank_001_alphafold2_multim...,NaN,실패: Vina 실행 에러(code=1) (PDBQT parsing error: f...,complex_2_unrelaxed_rank_001_alphafold2_multim...,complex_2_unrelaxed_rank_001_alphafold2_multim...,complex_2_unrelaxed_rank_001_alphafold2_multim...
3,complex_3_unrelaxed_rank_001_alphafold2_multim...,0.000,정상(점수=0.0),complex_3_unrelaxed_rank_001_alphafold2_multim...,complex_3_unrelaxed_rank_001_alphafold2_multim...,complex_3_unrelaxed_rank_001_alphafold2_multim...
4,complex_4_unrelaxed_rank_001_alphafold2_multim...,-2.899,정상,complex_4_unrelaxed_rank_001_alphafold2_multim...,complex_4_unrelaxed_rank_001_alphafold2_multim...,complex_4_unrelaxed_rank_001_alphafold2_multim...
5,complex_5_unrelaxed_rank_001_alphafold2_multim...,-4.275,정상,complex_5_unrelaxed_rank_001_alphafold2_multim...,complex_5_unrelaxed_rank_001_alphafold2_multim...,complex_5_unrelaxed_rank_001_alphafold2_multim...
6,complex_6_unrelaxed_rank_001_alphafold2_multim...,NaN,스킵: 체인 분리 실패(ValueError: /home/aisys/work/pipe...,NaN,NaN,NaN
7,complex_7_unrelaxed_rank_001_alphafold2_multim...,-3.866,정상,complex_7_unrelaxed_rank_001_alphafold2_multim...,complex_7_unrelaxed_rank_001_alphafold2_multim...,complex_7_unrelaxed_rank_001_alphafold2_multim...
8,complex_8_unrelaxed_rank_001_alphafold2_multim...,NaN,실패: Vina 실행 에러(code=1) (PDBQT parsing error: f...,complex_8_unrelaxed_rank_001_alphafold2_multim...,complex_8_unrelaxed_rank_001_alphafold2_multim...,complex_8_unrelaxed_rank_001_alphafold2_multim...
9,complex_9_unrelaxed_rank_001_alphafold2_multim...,-4.363,정상,complex_9_unrelaxed_rank_001_alphafold2_multim...,complex_9_unrelaxed_rank_001_alphafold2_multim...,complex_9_unrelaxed_rank_001_alphafold2_multim...
